<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/facevidcrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FaceVidCrop

### Notebook by [Eyal Gruss](https://eyalgruss.com), [@eyaler](twitter.com/eyaler)

Crop a video to center on a face

Optionally upscale with [GFPGAN](https://github.com/TencentARC/GFPGAN)

More notebooks: [github.com/eyaler/avatars4all](https://github.com/eyaler/avatars4all)

Shortcut here: [tfi.la/face](https://tfi.la/face)

In [ ]:
#@title Setup

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install git+https://github.com/ytdl-org/youtube-dl
!pip install imageio
!pip install git+https://github.com/1adrianb/face-alignment

%cd /content
!git clone --depth=1 https://github.com/TencentARC/GFPGAN
%cd /content/GFPGAN
!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan
%cd /content

In [ ]:
#@title Get Video

video_url = 'https://www.youtube.com/watch?v=YxWEYmU3eu0' #@param {type:"string"}

%cd /content
!rm -f /content/video.mp4
!rm -f /content/full_video.mp4
!youtube-dl -f "bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4[vcodec!*=av01]/mp4[vcodec!*=av01]/mp4" "$video_url" --merge-output-format mp4 -o /content/video.mp4
!cp /content/video.mp4 /content/full_video.mp4

In [ ]:
#@title Crop video to face

face_num = 0 #@param {type: 'integer'}
min_conf = .8 #@param {type: 'number'}
top_extend_frac = .7 #@param {type: 'number'}
bottom_extend_frac = .7 #@param {type: 'number'}
aspect_ratio = 1.33 #@param {type: 'number'}
start_seconds = 1 #@param {type: 'number'}
duration_seconds = 18 #@param {type: 'number'}
GFPGAN_model = '1.4' #@param ['1', '1.2', '1.3', '1.4', 'RestoreFormer']
GFPGAN_factor = 0 #@param {type: 'integer'}
temporal_mode = 'forward' #@param ['forward', 'reverse', 'forward + reverse']
output_filename = 'output.mp4' #@param {type: 'string'}

%cd /content
if duration_seconds:
  !ffmpeg -y -ss $start_seconds -t $duration_seconds -i /content/full_video.mp4 -f mp4 /content/video.mp4

import imageio.v3 as iio
from IPython.display import Video, clear_output
import face_alignment

fa = face_alignment.FaceAlignment(landmarks_type=1)
fps = iio.immeta('/content/video.mp4')['fps']

faces = []
good_faces = []
for im in iio.imiter('/content/video.mp4'):
  *_, bboxes = fa.get_landmarks_from_image(im, return_bboxes=True)
  found = False
  if min_conf:
    bboxes = [b for b in bboxes if b[-1] >= min_conf]
  if face_num:
    bboxes = sorted(bboxes, key=lambda p: (p[1], p[3], p[0], p[2]))[face_num - 1 : face_num]
  if bboxes:
    bbox = sorted(bboxes, key=lambda p: (p[2]-p[0]) * (p[3]-p[1]))[-1]
    faces.append(bbox[:-1])
    good_faces.append(faces[-1])
  else:
    faces.append(None)

median = sorted(good_faces, key=lambda p: (p[2]-p[0]) * (p[3]-p[1]))[len(good_faces) // 2]
top = (median[3] - median[1]) * (.5 + top_extend_frac)
h = top + (median[3] - median[1]) * (.5 + bottom_extend_frac)
if h > im.shape[0]:
  top = top / h * im.shape[0]
  h = im.shape[0]
h = int(h)
w = min(round(h * aspect_ratio), im.shape[1])

video = []
prev_face = good_faces[0]
for im, face in zip(iio.imiter('/content/video.mp4'), faces):
    x0, y0, x1, y1 = prev_face = prev_face if face is None else face
    x = (x0+x1) / 2
    y = (y0+y1) / 2

    x0 = max(x - w/2, 0)
    x1 = int(min(x0 + w, im.shape[1]))
    x0 = x1 - w

    y0 = max(y - top, 0)
    y1 = int(min(y0 + h, im.shape[0]))
    y0 = y1 - h

    video.append(im[y0 : y1, x0 : x1])

iio.imwrite('/content/crop.mp4', video, fps=fps)

if GFPGAN_factor:
  !rm -rf /content/in_frames
  !rm -rf /content/out_frames
  !mkdir -p /content/in_frames
  !mkdir -p /content/out_frames
  !ffmpeg -i "/content/crop.mp4" -compression_level 1 /content/in_frames/frame_%06d.png
  %cd /content/GFPGAN
  !python inference_gfpgan.py -i /content/in_frames -o /content/out_frames -v $GFPGAN_model -s $GFPGAN_factor
  %cd /content
  !ffmpeg -y -framerate $fps -i /content/out_frames/restored_imgs/frame_%06d.png -i /content/video.mp4 -c:v libx264 -c:a aac -map 0:v -map 1:a? -pix_fmt yuv420p -profile:v baseline -movflags +faststart "/content/$output_filename"
else:
  !ffmpeg -y -i /content/crop.mp4 -i /content/video.mp4 -c:v libx264 -c:a aac -map 0:v -map 1:a? -pix_fmt yuv420p -profile:v baseline -movflags +faststart "/content/$output_filename"

if temporal_mode.endswith('reverse'):
  reverse_filename = 'reverse_' + output_filename
  !ffmpeg -y -i "/content/$output_filename" -vf reverse -af areverse "/content/$reverse_filename"
  if temporal_mode == 'forward + reverse':
    !echo file "/content/$output_filename" > list.txt
    !echo file "/content/$reverse_filename" >> list.txt
    concat_filename = 'concat_' + output_filename
    !ffmpeg -y -f concat -safe 0 -i list.txt -c copy "/content/$concat_filename"
    !mv "/content/$concat_filename" "/content/$output_filename"
  else:
    !mv "/content/$reverse_filename" "/content/$output_filename"
clear_output()

Video('/content/' + output_filename, embed=True, html_attributes="autoplay controls loop")

In [ ]:
#@title Download

from google.colab import files
files.download('/content/' + output_filename)